# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [23]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,westport,41.1415,-73.3579,9.95,38,0,0.89,US,1731193277
1,1,grytviken,-54.2811,-36.5092,-4.63,73,11,4.63,GS,1731192876
2,2,adamstown,-25.0660,-130.1015,23.16,88,99,9.42,PN,1731193449
3,3,waitangi,-43.9535,-176.5597,12.95,92,96,4.55,NZ,1731192857
4,4,punta arenas,-53.1500,-70.9167,7.06,87,75,4.12,CL,1731192957


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [25]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    alpha = .5,
    frame_width = 800,
    frame_height = 600
    
)
# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df[(city_data_df['Wind Speed']<4.5) & (city_data_df['Cloudiness'] == 0) & (city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21)]

# Drop any rows with null values
filtered_df.dropna()

# Display sample data
filtered_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,tazacorte,28.6290,-17.9293,25.64,29,0,4.47,ES,1731193454
37,37,mar del plata,-38.0023,-57.5575,23.01,56,0,3.60,AR,1731193055
127,127,antonio enes,-16.2325,39.9086,23.15,87,0,0.91,MZ,1731193617
150,150,howrah,22.5892,88.3103,23.00,88,0,0.00,IN,1731193647
223,223,sompeta,18.9333,84.6000,22.69,83,0,3.11,IN,1731193741


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
6,tazacorte,ES,28.6290,-17.9293,29,
37,mar del plata,AR,-38.0023,-57.5575,56,
127,antonio enes,MZ,-16.2325,39.9086,87,
150,howrah,IN,22.5892,88.3103,88,
223,sompeta,IN,18.9333,84.6000,83,
262,khedbrahma,IN,24.0333,73.0500,40,
270,puerto del carmen,ES,28.9231,-13.6658,41,
299,chandbali,IN,20.7833,86.7667,79,
336,piton saint-leu,RE,-21.2196,55.3151,73,
370,timbuktu,ML,16.7735,-3.0074,11,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tazacorte - nearest hotel: App Leyma
mar del plata - nearest hotel: Hotel 3 de Abril
antonio enes - nearest hotel: Hotel Quirimbas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    alpha = .5,
    frame_width = 800,
    frame_height = 600,
    hover_cols = 'Hotel Name'
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)